In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.webdriver import ActionChains
import base64
import hashlib
from Crypto.Cipher import AES
from Crypto import Random
import time
import csv
fields = ['Title', 'Rating', 'Review 1', 'Review 2', 'Review 3', 'Description 1', 'Description 2']  
rows = []
browser = webdriver.Chrome()
pages = ["Furniture", "Mattresses"]
BLOCK_SIZE = 16
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)
unpad = lambda s: s[:-ord(s[len(s) - 1:])]
password = "globalizationfacilitator"
def encrypt(raw, password):
    private_key = hashlib.sha256(password.encode("utf-8")).digest()
    raw = pad(raw)
    iv = Random.new().read(AES.block_size)
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return base64.b64encode(iv + cipher.encrypt(raw))
def decrypt(enc, password):
    private_key = hashlib.sha256(password.encode("utf-8")).digest()
    enc = base64.b64decode(enc)
    iv = enc[:16]
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(enc[16:]))
for j in range(len(pages)):
    time.sleep(10)
    browser.get('http://www.yelp.com')
    time.sleep(10)
    elem = browser.find_element_by_id('find_desc')
    elem.send_keys(pages[j] + Keys.RETURN)
    for i in range(3):
        mylist = []
        try:
            time.sleep(10)
            link = browser.find_element_by_xpath("(//h3[@class=' heading--h5__09f24__FyKNw undefined'])[2]/ancestor::li" + (i + 1) * "/following-sibling::li" + "//a")
            link.click()
            time.sleep(10)
            mylist.append(encrypt(titlename, password))
        except:
            mylist.append(encrypt('Could not get title', password))
        try:
            stars = browser.find_element_by_xpath("//*[@id='wrap']/div[3]/yelp-react-root/div/div[3]/div[1]/div[1]/div/div/div[2]/div[1]/span/div")
            rating = stars.get_attribute("aria-label")
            mylist.append(encrypt(rating, password))
        except:
            mylist.append(encrypt('Could not get rating', password))
        for j in range(3):
            try:
                time.sleep(10)
                review = browser.find_element_by_xpath("//*[@id='wrap']/div[3]/yelp-react-root/div/div[4]/div/div/div[2]/div/div/div[1]/div/div[1]/div[2]/section[2]/div[2]/div/ul/li[" + str(j + 1) + "]/div//p/span")
                revinfo = review.get_attribute('innerText')
                mylist.append(encrypt(revinfo, password))
            except:
                mylist.append(encrypt('Could not get review ' + str(j + 1), password))
        try:
            time.sleep(10)
            desc = browser.find_element_by_xpath("//span[contains(text(), 'Read more')]")
            desc.click()
            time.sleep(10)
            desc2 = browser.find_element_by_xpath("//*[@id='modal-portal-container']/div/div/div/div/div/div[2]//p[1]")
            info = desc2.get_attribute('innerText')
            mylist.append(encrypt(info, password))
        except:
            mylist.append(encrypt('Could not get description 1', password))
        try:
            time.sleep(10)
            desc2 = browser.find_element_by_xpath("//*[@id='modal-portal-container']/div/div/div/div/div/div[2]//p[2]")
            info = desc2.get_attribute('innerText')
            mylist.append(encrypt(info, password))
        except:
            mylist.append(encrypt('Could not get description 2', password))
        rows.append(mylist)
        browser.back()
        time.sleep(10)
filename = "globalization_facilitator.csv"
with open(filename, 'w') as csvfile:  
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerow(fields)  
    csvwriter.writerows(rows)